In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GRU, Bidirectional, TimeDistributed, BatchNormalization, Embedding

from numpy import array
from keras.models import load_model
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings('ignore')

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
scl = preprocessing.StandardScaler()

#https://www.kaggle.com/mikhailhushchyn/dbscan-benchmark
#https://www.kaggle.com/mikhailhushchyn/hough-transform
def norm_points(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2 + y**2 + z**2)
    df['x2'] = x/r
    df['y2'] = y/r
    df['z2'] = z / np.sqrt(x**2 + y**2)
    return df


In [3]:
def append_to_csv(batch, csv_file):
    props = dict(encoding='utf-8', index=False)
    if not os.path.exists(csv_file):
        batch.to_csv(csv_file, **props)
    else:
        batch.to_csv(csv_file, mode='a', header=False, **props)

def delete_file_if_exists(filename):
    if os.path.exists(filename):
        os.remove(filename)

In [4]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [5]:
hits = pd.read_csv('../cache/hits_6488.csv')
# hits['new_pid'] = hits.particle_id.astype('str') + '_' + hits.event_id.astype('str') 
# hits = hits[hits.nhits >= 9]
hits = norm_points(hits)

In [6]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,px,py,pz,q,nhits,event_id,new_pid,x2,y2,z2
0,2,-72.4041,-1.15933,-1502.5,7,2,1,702569307170668544,-72.4163,-1.13906,...,-0.376872,-0.027696,-7.81030,1.0,14.0,6488,702569307170668544_6488,-0.048133,-0.000771,-20.748928
1,3,-56.7804,-8.25539,-1502.5,7,2,1,58552223994478592,-56.7717,-8.23860,...,-0.993228,-0.127344,-26.20750,-1.0,12.0,6488,58552223994478592_6488,-0.037763,-0.005490,-26.186271
2,4,-93.1412,-14.67800,-1502.5,7,2,1,1062854322422808576,-93.1464,-14.66960,...,-0.194728,0.000293,-3.03961,-1.0,16.0,6488,1062854322422808576_6488,-0.061869,-0.009750,-15.934772
3,6,-68.1201,-6.84073,-1502.5,7,2,1,833166824416739328,-68.1281,-6.81962,...,-0.170736,-0.037910,-3.82846,1.0,12.0,6488,833166824416739328_6488,-0.045291,-0.004548,-21.946251
4,7,-60.4104,-11.60050,-1502.5,7,2,1,887215586222800896,-60.3889,-11.61580,...,-0.359280,-0.049738,-8.85119,-1.0,11.0,6488,887215586222800896_6488,-0.040173,-0.007714,-24.425283


In [7]:
hits.shape

(121814, 28)

In [8]:
hits.particle_id.nunique()

10149

In [9]:
new_pid_list = list(set(hits.new_pid.values))

new_pid_count = list(range(len(new_pid_list)))
new_pid_dict = dict(zip(new_pid_list, new_pid_count))
    
    
hits['nid'] = hits['new_pid'].map(lambda x: new_pid_dict[x])
hits.sort_values(['nid', 'z'], inplace=True)

In [10]:
hits.tail()

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,...,py,pz,q,nhits,event_id,new_pid,x2,y2,z2,nid
96469,116061,-471.454,154.666,52.0000,13,6,933,216176218087620608,-471.462,154.651,...,-0.089198,0.021324,1.0,13.0,6488,216176218087620608_6488,-0.945000,0.310018,0.104802,10148
96119,115565,-473.197,157.971,52.2000,13,6,855,216176218087620608,-473.192,157.982,...,-0.089198,0.021324,1.0,13.0,6488,216176218087620608_6488,-0.943389,0.314939,0.104637,10148
96114,115559,-476.419,164.489,52.5429,13,6,854,216176218087620608,-476.431,164.469,...,-0.089198,0.021324,1.0,13.0,6488,216176218087620608_6488,-0.940152,0.324598,0.104249,10148
100945,123411,-493.809,433.838,83.2000,13,8,1213,216176218087620608,-493.831,433.819,...,-0.089198,0.021324,1.0,13.0,6488,216176218087620608_6488,-0.745305,0.654791,0.126576,10148
100942,123407,-489.842,445.385,84.4000,13,8,1212,216176218087620608,-489.841,445.386,...,-0.089198,0.021324,1.0,13.0,6488,216176218087620608_6488,-0.733944,0.667333,0.127482,10148


In [11]:
model = Sequential()
# model.add(BatchNormalization(input_shape=(1,2)))
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(1,3)))
# model.add(Bidirectional(LSTM(100, return_sequences=True)))
# model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Bidirectional(LSTM(100)))
# model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))

# model.add(Dense(100000, activation='relu'))

# model.add(Dropout(0.1))
model.add(Dense(10149, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [12]:
for n in tqdm(range(102)):
#     if n == 0:
#         continue
    w_start = n*100
    w_end = (n+1)*100
    if w_end > 10149:
        w_end = 10149
    hits1 = hits[(hits.nid >= w_start) & (hits.nid < w_end)]
   
    str1='../cache/checkpoint/checkpoint-6488-copy3-{}'.format(n)
    str2 = '{epoch:02d}.hdf5'
    filepath = str1 + '-' + str2
    checkpoint = ModelCheckpoint(filepath, verbose=0, save_best_only=False)
    callbacks_list = [checkpoint]

#     print(hits1.head())
    X_train = scl.fit_transform(hits1[['x', 'y', 'z']].values)
#     X_train = hits1[['x', 'y', 'z']].values
    
    y = hits1['nid'].values
    print(y)
#     print(len(y))
    print(X_train.shape)
    
    model.fit(X_train.reshape(X_train.shape[0], 1, 3), y, batch_size=32, epochs=3000, shuffle=False, verbose=0, 
          callbacks=callbacks_list)

  0%|          | 0/102 [00:00<?, ?it/s]

[ 0  0  0 ... 99 99 99]
(1174, 3)


  1%|          | 1/102 [1:02:09<104:38:15, 3729.66s/it]

[100 100 100 ... 199 199 199]
(1212, 3)


  2%|▏         | 2/102 [2:00:08<100:07:13, 3604.34s/it]

...

 81%|████████▏ | 83/102 [86:15:51<19:44:50, 3741.58s/it]

[8300 8300 8300 ... 8399 8399 8399]
(1210, 3)


 82%|████████▏ | 84/102 [87:18:36<18:42:33, 3741.87s/it]

[8400 8400 8400 ... 8499 8499 8499]
(1195, 3)


KeyboardInterrupt: 